### El área de Demand Planning requiere información con mayor frecuencia de las transacciones de venta y movimientos de mercadería realizadas. Para ello genera reportes  comerciales  a  través  del  área  de Reporting  con  diversos  KPIs,  indicadores segmentando por distintas categorías

##### [Link al modelo de datos](https://drive.google.com/file/d/1NNaDbxox1cj3vgA6THKbSNgf6pLe7ES-/view)

###### El modelo cuenta con las siguientes tablas:
- Clientes: Listado de los clientes dados de alta en el sistema de ventas.
- Empleados: Maestro de empleados, el mismo esta compuesto por el identificador, nombre, apellido y sucursal en la que trabaja.
- Locales: Maestro de sucursales compuesta por el identificador, nombre y tipo de local.
- Productos: Maestro de productos con su precio agrupados por familia de producto.
- Facturas: Tabla que registra todas las transacciones (ventas). Además contiene, la fecha de en que se realizó la operación, el empleado que hizo la venta, el cliente y la cantidad de productos vendidos

INTEGRANTES:
  

### Daniel Ortiz Aristizábal

In [0]:
USE curso.ventas

1. Generar un listado de la cantidad de productos vendidos por año de manera descendente.

In [0]:
SELECT
  YEAR(F.fecha_venta) AS ano,
  SUM(F.cantidad) AS cant_productos_vendidos
FROM facturas F
GROUP BY YEAR(F.fecha_venta)
ORDER BY cant_productos_vendidos DESC

ano,cant_productos_vendidos
2021,3348633
2022,3336752
2023,2391118
2020,966296


2. Top-5 de los empleados que menos vendieron según cantidad vendida, indicando apellido y nombre en un sólo campo. 

In [0]:
SELECT
  CONCAT(E.apellido, ' ', E.nombre) AS vendedor,
  SUM(F.cantidad) AS cant_productos_vendidos
FROM empleados E
INNER JOIN facturas F ON E.id_vendedor = F.vendedor
GROUP BY E.apellido, E.nombre
ORDER BY cant_productos_vendidos ASC
LIMIT 5

vendedor,cant_productos_vendidos
Pérez Jose,142741
González Maria,147177
Fernández Luis,149327
González Ramon,149346
García Ana,149925


 3. ¿Cuántos clientes compraron mes anterior ?

In [0]:
SELECT
  COUNT(DISTINCT cliente) AS num_clientes
FROM facturas
WHERE MONTH(fecha_venta) = MONTH(CURRENT_DATE() - INTERVAL 1 MONTH)
  AND YEAR(fecha_venta) = YEAR(CURRENT_DATE() - INTERVAL 1 MONTH)

num_clientes
0


In [0]:
SELECT MAX(fecha_venta) FROM facturas

MAX(fecha_venta)
2023-09-17


In [0]:
SELECT 
  COUNT(DISTINCT cliente) AS num_clientes
FROM facturas
WHERE 
  MONTH(fecha_venta) = MONTH((SELECT MAX(fecha_venta) FROM facturas) - INTERVAL 1 MONTH)
  AND YEAR(fecha_venta) = YEAR((SELECT MAX(fecha_venta) FROM facturas) - INTERVAL 1 MONTH)

num_clientes
1677


4. ¿Cuál fue el producto que se vendió mas en el año 2022? ¿A qué familia de producto pertenece?

In [0]:
SELECT
  P.nombre AS producto,
  P.familia,
  SUM(F.cantidad) AS cant_vendida
FROM facturas F
INNER JOIN productos P ON F.producto = P.id_producto
WHERE YEAR(F.fecha_venta) = 2022
GROUP BY P.nombre, P.familia
ORDER BY cant_vendida DESC
LIMIT 1

producto,familia,cant_vendida
Triángulo,Chocolates,57911


5. Siguiendo con el punto anterior ¿Y cuál fue el más rentable?

In [0]:
SELECT
  P.nombre AS producto,
  P.familia,
  SUM(F.cantidad) AS cant_vendida,
  P.precio_unitario,
  SUM(F.cantidad * P.precio_unitario) AS total_ingresos
FROM facturas F
INNER JOIN productos P ON F.producto = P.id_producto
WHERE YEAR(F.fecha_venta) = 2022
GROUP BY P.id_producto, P.nombre, P.familia, P.precio_unitario
ORDER BY total_ingresos DESC
LIMIT 1

producto,familia,cant_vendida,precio_unitario,total_ingresos
Häagen-Dazs,Helados,32433,246,7978518


6. Top-10 de sucursales según monto vendido, indicando el monto, ordenado de mayor a menor. El informe debe mostrar:
- Tipo de local
- Nombre del local
- Monto vendido

In [0]:
SELECT
  L.tipo AS tipo_local,
  L.nombre AS nombre_local,
  SUM(F.cantidad * P.precio_unitario) AS monto_vendido
FROM locales L
INNER JOIN empleados E ON L.id_sucursal = E.sucursal
INNER JOIN facturas F ON E.id_vendedor = F.vendedor
INNER JOIN productos P ON F.producto = P.id_producto
GROUP BY L.tipo, L.nombre
ORDER BY monto_vendido DESC
LIMIT 10

tipo_local,nombre_local,monto_vendido
Supermercado,Éxito La 33,171107614
Supermercado,Éxito Itagüí,130984032
Supermercado,Éxito Aranjuez,130839954
Supermercado,Éxito Junin,104728345
Comercio de cercania,Éxito La Ceja,87719090
Comercio de cercania,Éxito Indiana Mall,86263728
Supermercado,Éxito Gran Vía,86079524
Supermercado,Éxito Belén,65500936
Vecino,Éxito Del Este,64887456
Supermercado,Éxito Envigado Centro,63641985


7. Se detectaron ventas (facturas) realizadas por vendedores que no estan mas en la compañia (no estan en el maestro de empleados). Por lo tanto, nos solicitan un listado de dichos empleados con la cantidad de ventas (facturas). ¿Cuántos empleados son?

In [0]:
SELECT
  F.vendedor AS id_vendedor,
  COUNT(*) AS cant_facturas
FROM facturas F
LEFT JOIN empleados E ON F.vendedor = E.id_vendedor
WHERE E.id_vendedor IS NULL
GROUP BY F.vendedor
ORDER BY cant_facturas DESC

id_vendedor,cant_facturas
61,1608
62,1580
60,1580
59,1569
64,1558
65,1546
57,1513
63,1495
58,1473


In [0]:
SELECT 
  COUNT(DISTINCT F.vendedor) AS empleados_inexistentes
FROM facturas F
LEFT JOIN empleados E 
  ON F.vendedor = E.id_vendedor
WHERE E.id_vendedor IS NULL

empleados_inexistentes
9


Se identificaron 9 vendedores que registran ventas asociadas en la tabla facturas y que actualmente no existen en el maestro de empleados (tabla empleados)

8. Nos piden clasificar a los vendedores en funcion de su rendimiento (facturación) para el año actual.
- "Excelente" si el vendedor ha vendido por más de 10 millones de pesos en total.
- "Bueno" si el vendedor ha vendido entre 5 y 10 millones de pesos en total.
- "Regular" si el vendedor ha vendido menos de 5 millones de pesos en total.

In [0]:
SELECT
  CONCAT(E.nombre, ' ', E.apellido) AS vendedor,
  SUM(F.cantidad * P.precio_unitario) AS facturacion,
  CASE 
    WHEN SUM(F.cantidad * P.precio_unitario) > 10000000 THEN 'Excelente'
    WHEN SUM(F.cantidad * P.precio_unitario) BETWEEN 5000000 AND 10000000 THEN 'Bueno'
    WHEN SUM(F.cantidad * P.precio_unitario) < 5000000 THEN 'Regular'
  END AS rendimiento
FROM empleados E
INNER JOIN facturas F ON E.id_vendedor = F.vendedor
INNER JOIN productos P ON F.producto = P.id_producto
WHERE YEAR(F.fecha_venta) = 2023
GROUP BY E.nombre, E.apellido

vendedor,facturacion,rendimiento
Maria Álvarez,5289390,Bueno
Juan García,5309936,Bueno
Carlos Gómez,5877249,Bueno
Luis Fernández,4457867,Regular
Jose López,5216936,Bueno
Ana González,5319502,Bueno
Jorge García,5750229,Bueno
Ana Pérez,4485311,Regular
Luis López,5302007,Bueno
Ana Rodríguez,5226957,Bueno


9. Muestra el número total de facturas para cada vendedor que haya realizado más de 100 ventas el año anterior. Incluye el nombre del vendedor y la cantidad de facturas.

In [0]:
SELECT
  CONCAT(E.nombre, ' ', E.apellido) AS vendedor,
  COUNT(*) AS total_facturas
FROM facturas F
INNER JOIN empleados E ON F.vendedor = E.id_vendedor
WHERE YEAR(F.fecha_venta) = YEAR((SELECT MAX(fecha_venta) FROM facturas)) - 1
GROUP BY E.id_vendedor, E.nombre, E.apellido
HAVING COUNT(*) > 100
ORDER BY total_facturas DESC

vendedor,total_facturas
Luis López,566
Juan Sánchez,557
Jose Álvarez,552
Maria Martínez,552
Ramon Pérez,541
Ramon Martínez,537
Hector null,537
Carlos Gómez,534
Jose Sánchez,533
Ramon Sánchez,531


10. Generar un listado de los clientes que realizaron mas de 50 compras y que su edad sea mayor al premedio de edad del total de nuestra base de clientes. Ordenar el listado por edad de manera ascendente

In [0]:
SELECT
  CONCAT(C.nombre, ' ', C.apellido) AS cliente,
  YEAR(CURRENT_DATE()) - YEAR(C.fecha_nacimiento) AS edad,
  COUNT(*) AS compras
FROM clientes C
INNER JOIN facturas F ON C.id_cliente = F.cliente
GROUP BY C.id_cliente, C.nombre, C.apellido, C.fecha_nacimiento
HAVING COUNT(*) > 50 
  AND (YEAR(CURRENT_DATE()) - YEAR(C.fecha_nacimiento) > 
  (SELECT
    AVG(YEAR(CURRENT_DATE()) - YEAR(fecha_nacimiento))
  FROM clientes))
ORDER BY edad ASC

cliente,edad,compras
Samuel Benavent,51,55
Renata Palau,52,55
Pío Higueras,52,52
Florentina Carlos,53,53
Luis Mínguez,54,56
Valerio Arjona,55,52
Marcial Amor,55,53
Casandra Puente,55,52
Juan Pablo Ripoll,56,52
Rosaura Rocamora,56,52
